# How might we include `noarch: python` packages in a conda constructor?
Many of the pure-python packages on `conda-forge` are `python: noarch`, but the constructor project [isn't interested](https://github.com/conda/constructor/issues/96) in pursuing compatibility with them at this time for various, understandable reasons.

This is an exploration of packaging a non-trivial application (JupyterHub) as a constructor, initially started with [zero2one](https://github.com/bollwyvl/zero2one).

In [26]:
import subprocess, tarfile, tempfile, json, shutil, copy
from pathlib import Path

import jinja2
from ruamel_yaml import safe_dump, safe_load

from constructor.construct import parse
from constructor.conda_interface import cc_platform

In [2]:
def run(args):
    print("Running", "\n\t", " ".join(args))
    proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in iter(proc.stdout.readline, b''):
        print(line.decode('utf-8').rstrip())

# Build a `construct.yaml`

In [3]:
Path("construct").mkdir(exist_ok=True)

In [4]:
%%file construct/construct.yaml
{% set version = "0.9.0" %}

name: JupyterHubOne
version: {{ version }}

install_in_dependency_order: True

channels:
- https://conda.anaconda.org/conda-forge/
- https://repo.anaconda.com/pkgs/main/
    
post_install: post_install.sh

specs:
# this drives our version
- jupyterhub =={{ version }}
# always use most recent conda
- conda
# use node LTS for stability
- nodejs >=8,<9
- python >=3.6,<3.7
- ncurses <6.1

Writing construct/construct.yaml


In [5]:
%%file construct/post_install.sh
#!/usr/bin/env bash
echo "OK!"

Writing construct/post_install.sh


# Build a (broken) installer

In [6]:
run(["constructor",
     "construct",
     "--verbose",
     "--cache-dir", "./construct-cache"])

Running 
	 constructor construct --verbose --cache-dir ./construct-cache
platform: linux-64
conda packages download: /home/weg/Documents/projects/constructorthopaedic/construct-cache/linux-64
specs: ['jupyterhub ==0.9.0', 'conda', 'nodejs >=8,<9', 'python >=3.6,<3.7', 'ncurses <6.1']


name: JupyterHubOne
version: 0.9.0
cache download location: /home/weg/Documents/projects/constructorthopaedic/construct-cache/linux-64
platform: linux-64
number of package: 66
    defaults::python-3.6.5-hc3d631a_2
    conda-forge::ca-certificates-2018.4.16-0
    defaults::conda-env-2.6.0-h36134e3_1
    defaults::libgcc-ng-7.2.0-hdf63c60_3
    defaults::libstdcxx-ng-7.2.0-hdf63c60_3
    defaults::libffi-3.2.1-hd88cf55_4
    defaults::ncurses-6.0-h9df7e31_2
    defaults::nodejs-8.11.1-hf484d3e_0
    defaults::openssl-1.0.2o-h20670df_0
    defaults::tk-8.6.7-hc745277_3
    defaults::xz-5.2.4-h14c3975_4
    defaults::yaml-0.1.7-had09818_2
    conda-forge::zlib-1.2.11-h470a237_3
    conda-forge::configurable-

# Determine what was `noarch: python`

In [7]:
cached_packages = list((Path("construct-cache") / cc_platform).glob("*.bz2"))

noarch = {}

for tarpath in cached_packages:
    with tarfile.open(tarpath, mode="r:bz2") as tf:
        index = json.loads(tf.extractfile("info/index.json").read().decode("utf-8"))
        if index.get("noarch") == "python":
            noarch[index["name"]] = tarpath
print(noarch)

{'python-dateutil': PosixPath('construct-cache/linux-64/python-dateutil-2.7.3-py_0.tar.bz2'), 'async_generator': PosixPath('construct-cache/linux-64/async_generator-1.9-0.tar.bz2'), 'pyasn1': PosixPath('construct-cache/linux-64/pyasn1-0.4.3-py_0.tar.bz2'), 'attrs': PosixPath('construct-cache/linux-64/attrs-18.1.0-py_1.tar.bz2'), 'appdirs': PosixPath('construct-cache/linux-64/appdirs-1.4.3-py_1.tar.bz2'), 'constantly': PosixPath('construct-cache/linux-64/constantly-15.1.0-py_0.tar.bz2'), 'service_identity': PosixPath('construct-cache/linux-64/service_identity-17.0.0-py_0.tar.bz2')}


# Build a conda package containing those tarballs

In [8]:
original_construct = parse("./construct/construct.yaml", cc_platform)
original_construct

{'name': 'JupyterHubOne',
 'version': '0.9.0',
 'install_in_dependency_order': True,
 'channels': ['https://conda.anaconda.org/conda-forge/',
  'https://repo.anaconda.com/pkgs/main/'],
 'post_install': 'post_install.sh',
 'specs': ['jupyterhub ==0.9.0',
  'conda',
  'nodejs >=8,<9',
  'python >=3.6,<3.7',
  'ncurses <6.1']}

In [9]:
recipe_yaml = jinja2.Template("""
package:
    name: {{ package_name }}
    version: {{ version }}

source:
    path: ../packages

build:
    number: 0
    skip: true  # [not {{ cc_platform|replace("-", "") }}]
    script:
    - mkdir -p $PREFIX/share/{{ package_name }}/{{ cc_platform }}
    - cp *.bz2 $PREFIX/share/{{ package_name }}/{{ cc_platform }}/
    - conda index $PREFIX/share/{{ package_name }}/{{ cc_platform }}/
    - mkdir -p $PREFIX/share/{{ package_name }}/noarch
    - conda index $PREFIX/share/{{ package_name }}/noarch

requirements:
    run: []

about:
    summary: a conda channel of extra packages for {{ name }}
    license: Other
""").render(
    cc_platform=cc_platform,
    **original_construct,
    package_name="{}-extras".format(original_construct["name"].lower())
)
print(recipe_yaml)


package:
    name: jupyterhubone-extras
    version: 0.9.0

source:
    path: ../packages

build:
    number: 0
    skip: true  # [not linux64]
    script:
    - mkdir -p $PREFIX/share/jupyterhubone-extras/linux-64
    - cp *.bz2 $PREFIX/share/jupyterhubone-extras/linux-64/
    - conda index $PREFIX/share/jupyterhubone-extras/linux-64/
    - mkdir -p $PREFIX/share/jupyterhubone-extras/noarch
    - conda index $PREFIX/share/jupyterhubone-extras/noarch

requirements:
    run: []

about:
    summary: a conda channel of extra packages for JupyterHubOne
    license: Other


In [10]:
recipe = safe_load(recipe_yaml)
recipe

{'package': {'name': 'jupyterhubone-extras', 'version': '0.9.0'},
 'source': {'path': '../packages'},
 'build': {'number': 0,
  'skip': True,
  'script': ['mkdir -p $PREFIX/share/jupyterhubone-extras/linux-64',
   'cp *.bz2 $PREFIX/share/jupyterhubone-extras/linux-64/',
   'conda index $PREFIX/share/jupyterhubone-extras/linux-64/',
   'mkdir -p $PREFIX/share/jupyterhubone-extras/noarch',
   'conda index $PREFIX/share/jupyterhubone-extras/noarch']},
 'requirements': {'run': []},
 'about': {'summary': 'a conda channel of extra packages for JupyterHubOne',
  'license': 'Other'}}

In [11]:
out_packages = (Path(".") / "conda-bld")
#shutil.rmtree(out_packages)
out_packages.mkdir(exist_ok=True)

In [12]:
with tempfile.TemporaryDirectory() as td:
    tdp = Path(td)
    recipe_dir = tdp / "recipe"
    recipe_dir.mkdir()
    (recipe_dir / "meta.yaml").write_text(recipe_yaml)
    package_dir = tdp / "packages"
    package_dir.mkdir()
    for pkg in noarch.values():
        shutil.copy2(str(pkg), package_dir)
    run([
        "conda-build",
        "--output-folder", str(out_packages),
        str(recipe_dir)
    ])

Running 
	 conda-build --output-folder conda-bld /tmp/tmpjbgz99lx/recipe
Adding in variants from internal_defaults
INFO:conda_build.variants:Adding in variants from internal_defaults
Attempting to finalize metadata for jupyterhubone-extras
INFO:conda_build.metadata:Attempting to finalize metadata for jupyterhubone-extras
updating: python-dateutil-2.7.3-py_0.tar.bz2
updating: async_generator-1.9-0.tar.bz2
updating: pyasn1-0.4.3-py_0.tar.bz2
updating: attrs-18.1.0-py_1.tar.bz2
updating: appdirs-1.4.3-py_1.tar.bz2
updating: constantly-15.1.0-py_0.tar.bz2
updating: service_identity-17.0.0-py_0.tar.bz2
BUILD START: ['jupyterhubone-extras-0.9.0-0.tar.bz2']
Copying /tmp/tmpjbgz99lx/packages to /home/weg/Documents/projects/constructorthopaedic/envs/_build/conda-bld/jupyterhubone-extras_1530241615144/work
source tree in: /home/weg/Documents/projects/constructorthopaedic/envs/_build/conda-bld/jupyterhubone-extras_1530241615144/work

Resource usage statistics from building jupyterhubone-extras:
 

# Build a (less broken) Installer

In [13]:
new_construct = copy.deepcopy(original_construct)
new_construct["name"] += "Fixed"
new_construct.setdefault("exclude", []).extend(list(noarch))
new_construct.setdefault("channels", []).append(out_packages.resolve().as_uri())
new_construct["specs"].append(recipe["package"]["name"])
if new_construct.get("post_install") is None:
    new_construct["post_install"] = "install_extras.sh"
new_construct

{'name': 'JupyterHubOneFixed',
 'version': '0.9.0',
 'install_in_dependency_order': True,
 'channels': ['https://conda.anaconda.org/conda-forge/',
  'https://repo.anaconda.com/pkgs/main/',
  'file:///home/weg/Documents/projects/constructorthopaedic/conda-bld'],
 'post_install': 'post_install.sh',
 'specs': ['jupyterhub ==0.9.0',
  'conda',
  'nodejs >=8,<9',
  'python >=3.6,<3.7',
  'ncurses <6.1',
  'jupyterhubone-extras'],
 'exclude': ['python-dateutil',
  'async_generator',
  'pyasn1',
  'attrs',
  'appdirs',
  'constantly',
  'service_identity']}

In [25]:
shebang = "#!/usr/bin/env bash"

In [16]:
install_script = jinja2.Template("""
set -x
${PREFIX}/bin/conda install \
    --yes \
    --prefix ${PREFIX} \
    --offline \
    --force \
    --channel file://${PREFIX}/share/{{ package["name"] }} \
    {{ packages }}
""").render(**recipe, cc_platform=cc_platform, packages=" ".join(list(noarch.keys()))).strip()

In [17]:
with tempfile.TemporaryDirectory() as td:
    tdp = Path(td)
    shutil.copytree(Path("construct"), tdp / "construct")
    tdp_ctx = tdp / "construct"
    (tdp_ctx / "construct.yaml").write_text(
        safe_dump(new_construct, default_flow_style=False)
    )
    if original_construct.get("post_install"):
        script_path = tdp_ctx / original_construct.get("post_install")
        script_lines = script_path.read_text().split("\n")
        if script_lines[0].startswith("#!"):
            script_lines = [
                script_lines[0],
                install_script,
                *script_lines[1:]
            ]
        else:
            script_lines = [
                shebang,
                install_script, 
                *script_lines
            ]
        script_path.write_text("\n".join(script_lines))
    else:
        (tdp_ctx / new_construct["post_install"]).write_text("\n".join([
            shebang,
            install_script
        ]))
    run(["constructor",
         str(tdp_ctx),
         "--verbose",
         "--cache-dir", "./construct-cache"])

Running 
	 constructor /tmp/tmpgav80zyn/construct --verbose --cache-dir ./construct-cache
platform: linux-64
conda packages download: /home/weg/Documents/projects/constructorthopaedic/construct-cache/linux-64
specs: ['jupyterhub ==0.9.0', 'conda', 'nodejs >=8,<9', 'python >=3.6,<3.7', 'ncurses <6.1', 'jupyterhubone-extras']


name: JupyterHubOneFixed
version: 0.9.0
cache download location: /home/weg/Documents/projects/constructorthopaedic/construct-cache/linux-64
platform: linux-64
number of package: 60
    defaults::python-3.6.5-hc3d631a_2
    conda-forge::ca-certificates-2018.4.16-0
    defaults::conda-env-2.6.0-h36134e3_1
    file:///home/weg/Documents/projects/constructorthopaedic/conda-bld::jupyterhubone-extras-0.9.0-0
    defaults::libgcc-ng-7.2.0-hdf63c60_3
    defaults::libstdcxx-ng-7.2.0-hdf63c60_3
    defaults::libffi-3.2.1-hd88cf55_4
    defaults::ncurses-6.0-h9df7e31_2
    defaults::nodejs-8.11.1-hf484d3e_0
    defaults::openssl-1.0.2o-h20670df_0
    defaults::tk-8.6.7-hc74

# Test the constructor in docker
> TODO: generalize the pattern by tracking the output installer

In [18]:
Path("docker_test").mkdir(exist_ok=True)

In [19]:
%%file docker_test/Dockerfile
FROM centos:7
RUN yum install -y -qq bzip2
COPY JupyterHubOneFixed-0.9.0-Linux-x86_64.sh /tmp/
RUN bash /tmp/JupyterHubOneFixed-0.9.0-Linux-x86_64.sh -fbp /opt/jupyterhub
COPY startup.sh /tmp/
RUN chmod +x /tmp/startup.sh
CMD ["/tmp/startup.sh"]

Writing docker_test/Dockerfile


In [20]:
%%file docker_test/startup.sh
#!/usr/bin/env bash
set -ex
source /opt/jupyterhub/bin/activate /opt/jupyterhub
env | sort
jupyterhub

Writing docker_test/startup.sh


In [21]:
%%file docker-compose.yml
version: "3"
services:
  hub:
    build:
      context: docker_test
    ports:
      - 8081:8081
      - 8000:8000

Writing docker-compose.yml


In [22]:
!cp JupyterHubOneFixed-0.9.0-Linux-x86_64.sh docker_test/

In [23]:
run(["docker-compose", "build"])

Running 
	 docker-compose build
Building hub
Step 1/7 : FROM centos:7
 ---> 49f7960eb7e4
Step 2/7 : RUN yum install -y -qq bzip2
 ---> Using cache
 ---> d947fef0225d
Step 3/7 : COPY JupyterHubOneFixed-0.9.0-Linux-x86_64.sh /tmp/
 ---> 9d6430c11850
Removing intermediate container 72e1ae6d9e12
Step 4/7 : RUN bash /tmp/JupyterHubOneFixed-0.9.0-Linux-x86_64.sh -fbp /opt/jupyterhub
 ---> Running in 092da11385d0

PREFIX=/opt/jupyterhub
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.4.16-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: jupyterhubone-extras-0.9.0-0 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.0-h9df7e31_2 ...
installing: nodejs-8.11.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
in

In [24]:
run(["docker-compose", "up"])

Running 
	 docker-compose up
Recreating constructorthopaedic_hub_1 ...
Attaching to constructorthopaedic_hub_1
hub_1  | + source /opt/jupyterhub/bin/activate /opt/jupyterhub
hub_1  | ++ _CONDA_ROOT=/opt/jupyterhub
hub_1  | ++ . /opt/jupyterhub/etc/profile.d/conda.sh
hub_1  | +++ _CONDA_EXE=/opt/jupyterhub/bin/conda
hub_1  | +++ _CONDA_ROOT=/opt/jupyterhub
hub_1  | +++ _conda_set_vars
hub_1  | +++ '[' -n x ']'
hub_1  | +++ _CONDA_SHELL_FLAVOR=bash
hub_1  | +++ '[' -z x ']'
hub_1  | +++ '[' -z '' ']'
hub_1  | +++ PS1=
hub_1  | +++ '[' -z '' ']'
hub_1  | +++ export CONDA_SHLVL=0
hub_1  | +++ CONDA_SHLVL=0
hub_1  | ++ _conda_activate /opt/jupyterhub
hub_1  | ++ '[' -n '' ']'
hub_1  | ++ local ask_conda
hub_1  | +++ PS1=
hub_1  | +++ /opt/jupyterhub/bin/conda shell.posix activate /opt/jupyterhub
hub_1  | ++ ask_conda='PS1='\''(base) '\''
hub_1  | \export CONDA_DEFAULT_ENV='\''base'\''
hub_1  | \export CONDA_EXE='\''/opt/jupyterhub/bin/conda'\''
hub_1  | \export CONDA_PREFIX='\''/opt/jupyter

KeyboardInterrupt: 